In [119]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken pymupdf yfiles_jupyter_graphs

In [120]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.graph_transformers import LLMGraphTransformer
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
from typing import Tuple, List, Optional
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase
from langchain_core.runnables import ConfigurableField
from langchain_community.vectorstores import Neo4jVector
from google.colab import userdata
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import TokenTextSplitter
import uuid
import os
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import fitz
from pydantic import BaseModel, Field
from operator import itemgetter
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [121]:
NEO4J_URI="neo4j+s://4e590506.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="jil-tZeNk24kvNO7ia_O2EHQ7sO2Z6xc5gfolKcm2yU"
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [122]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

## Create  knowledge Graph

In [123]:
# Initialize a Neo4j graph instance
graph = Neo4jGraph()

In [124]:
# Initialize a text splitter with specified chunk size and overlap
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)

In [125]:
# Function to extract text from a given PDF file

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text


In [126]:
# Function to process all PDFs in a given folder, extract and split the text

def process_pdfs(pdf_folder):
    all_files_chunks = []

    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder, filename)
            print(f"Processing: {pdf_path}")

            text = extract_text_from_pdf(pdf_path)

            # Create a Document object containing the extracted text and metadata (like document ID and source file name)
            document = Document(
                page_content=text,
                metadata={
                    "document_id": str(uuid.uuid4()),
                    "source": filename,
                },
            )
            # Split the document's text into chunks
            chunks = text_splitter.split_documents([document])

            all_files_chunks.append(chunks)

    return all_files_chunks



In [127]:
pdf_folder = "/content/sample_data"
documents = process_pdfs(pdf_folder)

Processing: /content/sample_data/flowers.pdf


In [128]:
documents

[[Document(metadata={'document_id': 'defb7e20-43a5-4f3d-b2a6-6f17a8157400', 'source': 'flowers.pdf'}, page_content='A\xa0flower, also known as a\xa0bloom\xa0or\xa0blossom,[1]\xa0is the reproductive structure found in\xa0flowering plants\xa0(plants of the division\xa0Angiospermae). \nFlowers consist of a combination of vegetative organs –\xa0sepals\xa0that enclose and protect the developing flower. Petals attract pollinators, and \nreproductive organs that produce\xa0gametophytes, which in flowering plants produce\xa0gametes. The male gametophytes, which produce sperm, \nare enclosed within\xa0pollen grains\xa0produced in the\xa0anthers. The female gametophytes are contained within the\xa0ovules\xa0produced in the\xa0ovary.[2]\n[3]\xa0In some plants, multiple flowers occur singly on a\xa0pedicel\xa0(flower stalk), and some are arranged in a group (inflorescence) on \na\xa0peduncle\xa0(inflorescence stalk).[2]\n\n')]]

In [11]:
documents[0]

[Document(metadata={'document_id': '1486c4db-fd4e-4f2a-8d06-1dcb716b1912', 'source': 'MajorTrendsinGlobalEconomy.pdf'}, page_content='See discussions, stats, and author profiles for this publication at: https://www.researchgate.net/publication/368392470\nMajor Trends in Global Economy\nExperiment Findings · February 2023\nCITATIONS\n0\nREADS\n1,168\n2 authors, including:\ns. M. Saifee Islam\nThe KRF Center for Bangladesh and Global Affairs\n53 PUBLICATIONS\xa0\xa0\xa04 CITATIONS\xa0\xa0\xa0\nSEE PROFILE\nAll content following this page was uploaded by s. M. Saifee Islam on 10 February 2023.\nThe user has requested enhancement of the downloaded file.\n\nMajor Trends in Global Economy\nCBGA Assessment\nEditor\nDelwar Hossain\n\n \n \n \n \n \n \n \n \n \nMajor Trends in Global Economy \nCBGA Assessment \n \n \n \n \n \n \n \n \n \nEditor \nDelwar Hossain, PhD \n \n \n \n \n \n \n \n \n \n \n \n \nThe KRF Center for Bangladesh and Global Affairs (CBGA) \nHouse No. 12, Road No. 4, Dhanmond

In [129]:
# Initialize the LLM (Language Model)

llm=ChatOpenAI(temperature=0, model_name="gpt-4")

In [130]:
# Initialize the LLMGraphTransformer to convert documents into graph-friendly format

llm_transformer = LLMGraphTransformer(llm=llm)

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1373: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [156]:
# Convert the processed documents into graph documents
import time

x= time.time()
graph_documents = llm_transformer.convert_to_graph_documents([doc for sublist in documents for doc in sublist])
y = time.time()
print("time taken", y-x)

time taken 47.38400626182556


In [157]:
graph_documents

[GraphDocument(nodes=[Node(id='Flower', type='Structure', properties={}), Node(id='Bloom', type='Structure', properties={}), Node(id='Blossom', type='Structure', properties={}), Node(id='Flowering Plants', type='Organism', properties={}), Node(id='Angiospermae', type='Division', properties={}), Node(id='Sepals', type='Organ', properties={}), Node(id='Petals', type='Organ', properties={}), Node(id='Reproductive Organs', type='Organ', properties={}), Node(id='Gametophytes', type='Cell', properties={}), Node(id='Gametes', type='Cell', properties={}), Node(id='Male Gametophytes', type='Cell', properties={}), Node(id='Sperm', type='Cell', properties={}), Node(id='Pollen Grains', type='Structure', properties={}), Node(id='Anthers', type='Organ', properties={}), Node(id='Female Gametophytes', type='Cell', properties={}), Node(id='Ovules', type='Structure', properties={}), Node(id='Ovary', type='Organ', properties={}), Node(id='Pedicel', type='Structure', properties={}), Node(id='Inflorescence

In [158]:
# Add the graph documents to the Neo4j graph
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

## Test Cypher queries

In [25]:

default_cypher = "MATCH (d) - [r1] -> (o) - [r2] -> (n) WHERE o.id = 'Ukraine' RETURN  o, r1, d ,r2 LIMIT 25;"

In [26]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [27]:
def showGraph(cypher: str = default_cypher):
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [28]:
showGraph()

GraphWidget(layout=Layout(height='700px', width='100%'))

GraphWidget(layout=Layout(height='700px', width='100%'))

In [23]:
# graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

## retrieving data

In [136]:
#  Initializes a Neo4jVector object
from google.colab import userdata
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)


In [137]:
# container for storing a list of entity names that are identified from a text

class Entities(BaseModel):
    """Identifying information about entities."""
    names: List[str] = Field(
        ...,
        description="All entities that appear in the text",
    )


In [138]:
# extract data from the query
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)


/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1373: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [159]:
#  generate a full-text query based on the input
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [160]:

# Function to retrieve structured data by extracting named entities and querying the graph database for related information
def structured_retriever(question: str) -> str:
    result = ""
    document_ids = []
    entities = entity_chain.invoke({"question": question})

    for entity in entities.names:

        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
              LIMIT 2
            }
            CALL {
               WITH (node)
              MATCH (node) <- [:MENTIONS] - (d: Document)
              RETURN d.document_id AS document_id
              LIMIT 2
            }
            RETURN output, document_id LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )

        for el in response:
            print("gr_results", response)
            result += f"Output: {el['output']}\n"
            document_ids.append(el['document_id'])

    return result.strip(), document_ids

In [162]:

# Function to retrieve unstructured data based on a query, filtering the results based on relevance score >= 0.7

def retrieve_unstructured_data(question: str):
    search_results = vector_index.similarity_search_with_relevance_scores(question, k=3)
    print("vc_results", search_results)
    filtered_results = [doc for doc, score in search_results if score >= 0.7]
    # print("filtered_results", filtered_results)
    # print("filtered_results", len(filtered_results))

    document_ids = [el.metadata.get('document_id', 'N/A') for el in filtered_results]
    # print("document_ids", document_ids)
    page_contents = [doc.page_content for doc in filtered_results]

    return page_contents, document_ids

In [163]:

# Function to combine structured and unstructured data from different sources into a final result
def retriever(question: str):
    unstructured_data, document_ids_vg = retrieve_unstructured_data(question)
    structured_data, document_ids_kg = structured_retriever(question)

    document_ids = List [set( document_ids_vg + document_ids_kg )]

    final_data = f"""Structured data: {structured_data} Unstructured data: {"#Document ".join(unstructured_data)}"""

    return final_data, document_ids

In [164]:
# Template for the final response generation, incorporating context, question, and expected structured answer
template = """
You are a knowledgeable assistant with expertise in answering questions based on the provided context. Your task is to read the context carefully, understand the information, and provide a detailed and well-explained answer.

### Context:
{context}

### Question:
{question}

Provide the response as follows:

## Answer:
Provide a clear, concise, and accurate response based on the context. Then, explain your reasoning behind the answer. If the answer is not explicitly stated, infer based on the available information and justify your reasoning. Be sure to explain the relevant details, relationships, or assumptions used to arrive at your answer. Your explanation should help the user understand the reasoning behind your response.

"""
prompt = ChatPromptTemplate.from_template(template)


In [170]:
# Store the query to be used throughout the process
query_var = "query = what are the negative impacts of the Russian war?"

In [166]:
# Define the processing step to retrieve structured and unstructured data from retriver
processed = RunnableLambda(lambda d: retriever(d["question"]))

In [167]:
# Extract the retrieved data and document IDs from the processed results rom retriver
retrieved_data = processed | itemgetter(0)
retrieved_doc_ids = processed | itemgetter(1)

In [168]:
# Set up the final chain to format the response
chain = (
    {
        "context": retrieved_data,
        "question": itemgetter("question"),
    }
    | prompt
    | llm
    | StrOutputParser()
    | (lambda response: f"{response} | Document IDs: {retrieved_doc_ids.invoke({'question': query_var})}")
)

In [173]:
# Execute the chain to generate the result and print it
result = chain.invoke({"question": query_var})

print(result)

vc_results [(Document(metadata={'source': 'TheRusso-UkrainianWar.pdf', 'document_id': 'ca75bf67-f2df-4e57-bbe2-0a65db96889e'}, page_content="\ntext:  War and boundary formation: The Russo-\nUkrainian war and European integration. Ukraine’s Thorny Path to the EU, 19. \nHanley, H. W., Lu, Y., & Pan, J. (2025). Across the firewall: Foreign media’s role in shaping \nChinese social media narratives on the Russo-Ukrainian War. Proceedings of the National \nAcademy of Sciences, 122(1), e2420607122. \nIvanović, J., Vincze, O., Jevtić, M., Szabó, Z., Csertő, I., Choi, S. Y., & Liu, J. H. (2025). \nBetween east and west, between past and future: The effects of exclusive historical victimhood \non geopolitical attitudes in Hungary and Serbia. British Journal of Social Psychology. \n \nSaaida, M. (2023). China's Foreign Policy towards the Russian-Ukrainian Crisis. \nSaaida, M. (2023). Russia in Africa: A New Era of Multi-track Relationships. \nSaaida, M. (2024). Canned citations in scientific rese

gr_results [{'output': 'War Between Russia And Ukraine - IMPACTS -> Macroeconomics', 'document_id': 'ba35261f-4209-47ea-862e-5276c19f99f6'}]


vc_results [(Document(metadata={'source': 'TheRusso-UkrainianWar.pdf', 'document_id': 'ca75bf67-f2df-4e57-bbe2-0a65db96889e'}, page_content="\ntext:  War and boundary formation: The Russo-\nUkrainian war and European integration. Ukraine’s Thorny Path to the EU, 19. \nHanley, H. W., Lu, Y., & Pan, J. (2025). Across the firewall: Foreign media’s role in shaping \nChinese social media narratives on the Russo-Ukrainian War. Proceedings of the National \nAcademy of Sciences, 122(1), e2420607122. \nIvanović, J., Vincze, O., Jevtić, M., Szabó, Z., Csertő, I., Choi, S. Y., & Liu, J. H. (2025). \nBetween east and west, between past and future: The effects of exclusive historical victimhood \non geopolitical attitudes in Hungary and Serbia. British Journal of Social Psychology. \n \nSaaida, M. (2023). China's Foreign Policy towards the Russian-Ukrainian Crisis. \nSaaida, M. (2023). Russia in Africa: A New Era of Multi-track Relationships. \nSaaida, M. (2024). Canned citations in scientific rese

gr_results [{'output': 'War Between Russia And Ukraine - IMPACTS -> Macroeconomics', 'document_id': 'ba35261f-4209-47ea-862e-5276c19f99f6'}]
The negative impacts of the Russo-Ukrainian War, as outlined in the context, can be categorized into several areas:

1. Economic Repercussions: The war has had severe impacts on both nations and the broader global economy. Ukraine has suffered economic losses, including infrastructural damage, disrupted trade, and declining GDP. Western sanctions on Russia have also had mixed efficacy in curbing aggression while highlighting the economic resilience demonstrated by the Kremlin. The war has also had ripple effects on global energy markets, supply chains, and inflation.

2. Military Strategies and Security Implications: The war has led to concerns about escalation and the potential for broader conflict. The operational strategies employed by both sides, including the role of hybrid warfare, cyberattacks, and drone technology, have also been a cause f